# merge_shuffle_dataset

Merges (and shuffles) together multiple datasets in SP format.

In [1]:
import random
import h5py as h5
import numpy as np

# Modifying sys.path to be able to load project packages
import sys
sys.path.append('../')

# Load project packages
from utils.h5_tools import write_batch

In [2]:
out = 'C:/data/datasets/combined.h5'
batch_size = 100000
report_step = 100

seed = 42

n_samples = 400
n_channels = 3

label_aliases = {
                    0: 'p_wave',
                    1: 's_wave',
                    2: 'noise'
                }

`datasets` is a list of dictionaries, each dictionary representing dataset for the merge.

Dataset dictionary fields:
- `data_key` - data key, default "X"
- `label_key` - label key, default "Y", set it to `False` if you do not want to use labels (must specify `force_label`)
- `path` - path to the dataset file
- `normalize` - boolean, dataset will be normalized by maximum absolute value within one record is set to `True`
- `force_label` - if set will assign set value as label, regardless of original label value
- `select_label` - if set as value (numerical value), will use only that label value for the merge
- `length` - if set as integer value, will only select specified amount of entries from the dataset

In [13]:
datasets = [
    {
        'path': 'C:/data/datasets/stead_converted.h5',
        'normalize': True,
        'select_label': 0,
        'length': 158_414,
    },
    {
        'path': 'C:/data/datasets/stead_converted.h5',
        'normalize': True,
        'select_label': 1,
        'length': 158_414,
    },
    {
        'path': 'C:/data/datasets/false_positives_4s.h5',
        'normalize': True,
        'data_key': 'false-positives',
        'label_key': False,
        'force_label': 2
    },
    {
        'path': 'C:/data/datasets/meier_converted_noise.h5',
        'normalize': True,
        'length': 39_603,
    },
    {
        'path': 'C:/data/datasets/stead_converted.h5',
        'normalize': True,
        'select_label': 2,
        'length': 39_603,
    }
]

In [14]:
# Process default values and errors
for x in datasets:
    if 'select_label' in x and x['select_label'] not in label_aliases:
        raise AttributeError(f'Dataset {x["path"]} specified selection label {x["select_label"]} is not in label_aliases!')
    if 'label_key' in x and not x['label_key'] and 'force_label' not in x:
        raise AttributeError(f'Dataset {x["path"]}: provide either "label_key" or "force_label"')
    if 'data_key' not in x:
        x['data_key'] = 'X'
    if 'label_key' not in x:
        x['label_key'] = 'Y'

## Open datasets

In [15]:
actual_total_length = 0
total_length = 0

print('Datasets info: ')
for x in datasets:
    
    x['file'] = h5.File(x['path'], 'r')
    x['X'] = x['file'][x['data_key']]
    if x['label_key']:
        x['Y'] = x['file'][x['label_key']]
        x['actual_length'] = x['Y'].shape[0]
    else:
        x['actual_length'] = x['X'].shape[0]
    if 'length' not in x or x['length'] == -1:
        x['length'] = x['actual_length']
    elif x['length'] > x['actual_length']:
        raise AttributeError(f'Dataset {x["path"]} specified selection length '\
                             f'{x["length"]} is larger than actual dataset length {x["actual_length"]}')
        
    actual_total_length += x['actual_length']
    total_length += x['length']
    
    labels_count = {}
    if x['label_key']:
        for k in label_aliases:
            labels_count[k] = x['Y'][x['Y'][:] == k].shape[0]
    else:
        labels_count[x['force_label']] = x['length']
        
    if 'select_label' in x and labels_count[x['select_label']] < x['length']:
        raise AttributeError(f'Dataset {x["path"]} specified selection length '\
                             f'{x["length"]} is larger than actual label {label_aliases[x["select_label"]]} '\
                             f' items count {labels_count[x["select_label"]]}')
        
    print(f'\ndataset "{x["path"]}:"')
    print('  --data info--')
    print(f'\t---length: {x["actual_length"]}')
    for key, count in labels_count.items():
        print(f'\t---{label_aliases[key]} count: {count}')
        
    print(' --selection--')
    print(f'\t---selection length: {x["length"]}')
    if 'select_label' in x:
        print(f'\t---select label: {label_aliases[x["select_label"]]}')
    else:
          print(f'\t---select label: all')
    if 'force_label' in x:
        print(f'\t---force_label: {x["force_label"]}')
    
    print('  --data processing--')
    if x['normalize']:
        print('\t---normalize!')
        
print(f'Total length: {total_length}')

Datasets info: 
dataset "C:/data/datasets/stead_converted.h5:"
  --data info--
	---length: 2531314
	---p_wave count: 1030231
	---s_wave count: 1030231
	---noise count: 470852
 --selection--
	---selection length: 158414
	---select label: p_wave
  --data processing--
	---normalize!
dataset "C:/data/datasets/stead_converted.h5:"
  --data info--
	---length: 2531314
	---p_wave count: 1030231
	---s_wave count: 1030231
	---noise count: 470852
 --selection--
	---selection length: 158414
	---select label: s_wave
  --data processing--
	---normalize!
dataset "C:/data/datasets/false_positives_4s.h5:"
  --data info--
	---length: 79207
	---noise count: 79207
 --selection--
	---selection length: 79207
	---select label: all
	---force_label: 2
  --data processing--
	---normalize!
dataset "C:/data/datasets/meier_converted_noise.h5:"
  --data info--
	---length: 945571
	---p_wave count: 0
	---s_wave count: 0
	---noise count: 945571
 --selection--
	---selection length: 39603
	---select label: all
  --data 

## Generate and shuffle indexes

In [5]:
np.random.seed(seed)
new_labels = np.zeros((total_length, 2), dtype=int)  # [:, 0] - indexes inside the set, [:, 1] - set index

start = 0
for i, x in enumerate(datasets):
    
    end = start + x['length']
    new_labels[start:end, 0] = np.arange(x['length'])
    new_labels[start:end, 1] = np.full(x['length'], i, dtype=int)
    
np.random.shuffle(new_labels)

## Convert

In [6]:
batch_X = np.zeros((batch_size, n_samples, n_channels))
batch_Y = np.zeros(batch_size, dtype=int)

current_idx = 0
total = 0

for i in range(total_length):
    
    d_point = new_labels[i, 0]
    
    d_data = datasets[new_labels[i, 1]]
    X = d_data['X'][d_point]
    label = int(d_data['Y'][d_point])
    
    if d_data['force_label'] is not False:
        label = int(d_data['force_label'])
        
    if d_data['normalize']:
        a_max = np.max(np.abs(X))
        X /= a_max
        
    
    batch_X[current_idx] = X
    batch_Y[current_idx] = label
    current_idx += 1
    total += 1

    if current_idx >= batch_size:
        write_batch(out, 'X', batch_X)
        write_batch(out, 'Y', batch_Y)
        print(f'---Batch written to: {out}.. Saved total: {total} traces.')
        current_idx = 0
        
    if not total%report_step:
        print(f'\tConverted {total} traces.')
        
if current_idx != 0:
    write_batch(out, 'X', batch_X[:current_idx])
    write_batch(out, 'Y', batch_Y[:current_idx])
    
for x in datasets:
    x['file'].close()

	Converted 100 traces.
	Converted 200 traces.
	Converted 300 traces.
	Converted 400 traces.
	Converted 500 traces.
	Converted 600 traces.
	Converted 700 traces.
	Converted 800 traces.
	Converted 900 traces.
	Converted 1000 traces.
	Converted 1100 traces.
	Converted 1200 traces.
	Converted 1300 traces.
	Converted 1400 traces.
	Converted 1500 traces.
	Converted 1600 traces.
	Converted 1700 traces.
	Converted 1800 traces.
	Converted 1900 traces.
	Converted 2000 traces.
	Converted 2100 traces.
	Converted 2200 traces.
	Converted 2300 traces.
	Converted 2400 traces.
	Converted 2500 traces.
	Converted 2600 traces.
	Converted 2700 traces.
	Converted 2800 traces.
	Converted 2900 traces.
	Converted 3000 traces.
	Converted 3100 traces.
	Converted 3200 traces.
	Converted 3300 traces.
	Converted 3400 traces.
	Converted 3500 traces.
	Converted 3600 traces.
	Converted 3700 traces.
	Converted 3800 traces.
	Converted 3900 traces.
	Converted 4000 traces.
	Converted 4100 traces.
	Converted 4200 traces.
	

	Converted 33400 traces.
	Converted 33500 traces.
	Converted 33600 traces.
	Converted 33700 traces.
	Converted 33800 traces.
	Converted 33900 traces.
	Converted 34000 traces.
	Converted 34100 traces.
	Converted 34200 traces.
	Converted 34300 traces.
	Converted 34400 traces.
	Converted 34500 traces.
	Converted 34600 traces.
	Converted 34700 traces.
	Converted 34800 traces.
	Converted 34900 traces.
	Converted 35000 traces.
	Converted 35100 traces.
	Converted 35200 traces.
	Converted 35300 traces.
	Converted 35400 traces.
	Converted 35500 traces.
	Converted 35600 traces.
	Converted 35700 traces.
	Converted 35800 traces.
	Converted 35900 traces.
	Converted 36000 traces.
	Converted 36100 traces.
	Converted 36200 traces.
	Converted 36300 traces.
	Converted 36400 traces.
	Converted 36500 traces.
	Converted 36600 traces.
	Converted 36700 traces.
	Converted 36800 traces.
	Converted 36900 traces.
	Converted 37000 traces.
	Converted 37100 traces.
	Converted 37200 traces.
	Converted 37300 traces.


	Converted 66200 traces.
	Converted 66300 traces.
	Converted 66400 traces.
	Converted 66500 traces.
	Converted 66600 traces.
	Converted 66700 traces.
	Converted 66800 traces.
	Converted 66900 traces.
	Converted 67000 traces.
	Converted 67100 traces.
	Converted 67200 traces.
	Converted 67300 traces.
	Converted 67400 traces.
	Converted 67500 traces.
	Converted 67600 traces.
	Converted 67700 traces.
	Converted 67800 traces.
	Converted 67900 traces.
	Converted 68000 traces.
	Converted 68100 traces.
	Converted 68200 traces.
	Converted 68300 traces.
	Converted 68400 traces.
	Converted 68500 traces.
	Converted 68600 traces.
	Converted 68700 traces.
	Converted 68800 traces.
	Converted 68900 traces.
	Converted 69000 traces.
	Converted 69100 traces.
	Converted 69200 traces.
	Converted 69300 traces.
	Converted 69400 traces.
	Converted 69500 traces.
	Converted 69600 traces.
	Converted 69700 traces.
	Converted 69800 traces.
	Converted 69900 traces.
	Converted 70000 traces.
	Converted 70100 traces.


	Converted 99000 traces.
	Converted 99100 traces.
	Converted 99200 traces.
	Converted 99300 traces.
	Converted 99400 traces.
	Converted 99500 traces.
	Converted 99600 traces.
	Converted 99700 traces.
	Converted 99800 traces.
	Converted 99900 traces.
---Batch written to: C:/data/datasets/combined.h5.. Saved total: 100000 traces.
	Converted 100000 traces.
	Converted 100100 traces.
	Converted 100200 traces.
	Converted 100300 traces.
	Converted 100400 traces.
	Converted 100500 traces.
	Converted 100600 traces.
	Converted 100700 traces.
	Converted 100800 traces.
	Converted 100900 traces.
	Converted 101000 traces.
	Converted 101100 traces.
	Converted 101200 traces.
	Converted 101300 traces.
	Converted 101400 traces.
	Converted 101500 traces.
	Converted 101600 traces.
	Converted 101700 traces.
	Converted 101800 traces.
	Converted 101900 traces.
	Converted 102000 traces.
	Converted 102100 traces.
	Converted 102200 traces.
	Converted 102300 traces.
	Converted 102400 traces.
	Converted 102500 tr

	Converted 130300 traces.
	Converted 130400 traces.
	Converted 130500 traces.
	Converted 130600 traces.
	Converted 130700 traces.
	Converted 130800 traces.
	Converted 130900 traces.
	Converted 131000 traces.
	Converted 131100 traces.
	Converted 131200 traces.
	Converted 131300 traces.
	Converted 131400 traces.
	Converted 131500 traces.
	Converted 131600 traces.
	Converted 131700 traces.
	Converted 131800 traces.
	Converted 131900 traces.
	Converted 132000 traces.
	Converted 132100 traces.
	Converted 132200 traces.
	Converted 132300 traces.
	Converted 132400 traces.
	Converted 132500 traces.
	Converted 132600 traces.
	Converted 132700 traces.
	Converted 132800 traces.
	Converted 132900 traces.
	Converted 133000 traces.
	Converted 133100 traces.
	Converted 133200 traces.
	Converted 133300 traces.
	Converted 133400 traces.
	Converted 133500 traces.
	Converted 133600 traces.
	Converted 133700 traces.
	Converted 133800 traces.
	Converted 133900 traces.
	Converted 134000 traces.
	Converted 1

...

	Converted 8104400 traces.
	Converted 8104500 traces.
	Converted 8104600 traces.
	Converted 8104700 traces.
	Converted 8104800 traces.
	Converted 8104900 traces.
	Converted 8105000 traces.
	Converted 8105100 traces.
	Converted 8105200 traces.
	Converted 8105300 traces.
	Converted 8105400 traces.
	Converted 8105500 traces.
	Converted 8105600 traces.
	Converted 8105700 traces.
	Converted 8105800 traces.
	Converted 8105900 traces.
	Converted 8106000 traces.
	Converted 8106100 traces.
	Converted 8106200 traces.
	Converted 8106300 traces.
	Converted 8106400 traces.
	Converted 8106500 traces.
	Converted 8106600 traces.
	Converted 8106700 traces.
	Converted 8106800 traces.
	Converted 8106900 traces.
	Converted 8107000 traces.
	Converted 8107100 traces.
	Converted 8107200 traces.
	Converted 8107300 traces.
	Converted 8107400 traces.
	Converted 8107500 traces.
	Converted 8107600 traces.
	Converted 8107700 traces.
	Converted 8107800 traces.
	Converted 8107900 traces.
	Converted 8108000 traces.
	

	Converted 8134800 traces.
	Converted 8134900 traces.
	Converted 8135000 traces.
	Converted 8135100 traces.
	Converted 8135200 traces.
	Converted 8135300 traces.
	Converted 8135400 traces.
	Converted 8135500 traces.
	Converted 8135600 traces.
	Converted 8135700 traces.
	Converted 8135800 traces.
	Converted 8135900 traces.
	Converted 8136000 traces.
	Converted 8136100 traces.
	Converted 8136200 traces.
	Converted 8136300 traces.
	Converted 8136400 traces.
	Converted 8136500 traces.
	Converted 8136600 traces.
	Converted 8136700 traces.
	Converted 8136800 traces.
	Converted 8136900 traces.
	Converted 8137000 traces.
	Converted 8137100 traces.
	Converted 8137200 traces.
	Converted 8137300 traces.
	Converted 8137400 traces.
	Converted 8137500 traces.
	Converted 8137600 traces.
	Converted 8137700 traces.
	Converted 8137800 traces.
	Converted 8137900 traces.
	Converted 8138000 traces.
	Converted 8138100 traces.
	Converted 8138200 traces.
	Converted 8138300 traces.
	Converted 8138400 traces.
	

	Converted 8165200 traces.
	Converted 8165300 traces.
	Converted 8165400 traces.
	Converted 8165500 traces.
	Converted 8165600 traces.
	Converted 8165700 traces.
	Converted 8165800 traces.
	Converted 8165900 traces.
	Converted 8166000 traces.
	Converted 8166100 traces.
	Converted 8166200 traces.
	Converted 8166300 traces.
	Converted 8166400 traces.
	Converted 8166500 traces.
	Converted 8166600 traces.
	Converted 8166700 traces.
	Converted 8166800 traces.
	Converted 8166900 traces.
	Converted 8167000 traces.
	Converted 8167100 traces.
	Converted 8167200 traces.
	Converted 8167300 traces.
	Converted 8167400 traces.
	Converted 8167500 traces.
	Converted 8167600 traces.
	Converted 8167700 traces.
	Converted 8167800 traces.
	Converted 8167900 traces.
	Converted 8168000 traces.
	Converted 8168100 traces.
	Converted 8168200 traces.
	Converted 8168300 traces.
	Converted 8168400 traces.
	Converted 8168500 traces.
	Converted 8168600 traces.
	Converted 8168700 traces.
	Converted 8168800 traces.
	

	Converted 8195600 traces.
	Converted 8195700 traces.
	Converted 8195800 traces.
	Converted 8195900 traces.
	Converted 8196000 traces.
	Converted 8196100 traces.
	Converted 8196200 traces.
	Converted 8196300 traces.
	Converted 8196400 traces.
	Converted 8196500 traces.
	Converted 8196600 traces.
	Converted 8196700 traces.
	Converted 8196800 traces.
	Converted 8196900 traces.
	Converted 8197000 traces.
	Converted 8197100 traces.
	Converted 8197200 traces.
	Converted 8197300 traces.
	Converted 8197400 traces.
	Converted 8197500 traces.
	Converted 8197600 traces.
	Converted 8197700 traces.
	Converted 8197800 traces.
	Converted 8197900 traces.
	Converted 8198000 traces.
	Converted 8198100 traces.
	Converted 8198200 traces.
	Converted 8198300 traces.
	Converted 8198400 traces.
	Converted 8198500 traces.
	Converted 8198600 traces.
	Converted 8198700 traces.
	Converted 8198800 traces.
	Converted 8198900 traces.
	Converted 8199000 traces.
	Converted 8199100 traces.
	Converted 8199200 traces.
	

	Converted 8225700 traces.
	Converted 8225800 traces.
	Converted 8225900 traces.
	Converted 8226000 traces.
	Converted 8226100 traces.
	Converted 8226200 traces.
	Converted 8226300 traces.
	Converted 8226400 traces.
	Converted 8226500 traces.
	Converted 8226600 traces.
	Converted 8226700 traces.
	Converted 8226800 traces.
	Converted 8226900 traces.
	Converted 8227000 traces.
	Converted 8227100 traces.
	Converted 8227200 traces.
	Converted 8227300 traces.
	Converted 8227400 traces.
	Converted 8227500 traces.
	Converted 8227600 traces.
	Converted 8227700 traces.
	Converted 8227800 traces.
	Converted 8227900 traces.
	Converted 8228000 traces.
	Converted 8228100 traces.
	Converted 8228200 traces.
	Converted 8228300 traces.
	Converted 8228400 traces.
	Converted 8228500 traces.
	Converted 8228600 traces.
	Converted 8228700 traces.
	Converted 8228800 traces.
	Converted 8228900 traces.
	Converted 8229000 traces.
	Converted 8229100 traces.
	Converted 8229200 traces.
	Converted 8229300 traces.
	